0. This notebook is the continuation to the preprocessing.ipynb notebook.

1. Import all neccessary modules 

In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

2. Import X and y

In [2]:
import torchvision
from torchvision import transforms
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image

Code that loads the preprocessed npy arrays from the kosti dataset, extract frames and mscoco rows. Save for processing! This array contains all 3 variables at once.

In [163]:
y = np.load('/mnt/c/Users/janos/OneDrive/Desktop/Master_thesis/train_con.npy')

In [164]:
y

array([[5, 3, 9],
       [6, 4, 7],
       [7, 8, 8],
       ...,
       [5, 4, 6],
       [6, 7, 6],
       [5, 5, 6]])

In [165]:
y_ = y[0:100]

In [166]:
y_frames = y[14276:14376]

In [168]:
y_.shape, y_frames.shape

((100, 3), (100, 3))

In [170]:
y_all = np.concatenate((y_,y_frames))

NEW APPROACH:

Fit CNN on X

1. Read `n-rows`from X.

In [144]:
X = np.load('/mnt/c/Users/janos/OneDrive/Desktop/Master_thesis/X_npy.npy')
frames = np.load('/mnt/c/Users/janos/OneDrive/Desktop/Master_thesis/frames_npy.npy')

In [150]:
X_100 = X[0:100,:]

In [154]:
frames_100 = frames[0:100,:]

In [155]:
X_100.shape, frames_100.shape

((100, 1, 224, 224), (100, 1, 224, 224))

In [159]:
X_con = np.concatenate((X_100,frames_100))

In [171]:
X_con.shape, y_all.shape

((200, 1, 224, 224), (200, 3))

In [172]:
X_train,X_test,y_train,y_test = train_test_split(X_con,y_all,test_size=0.2,random_state=42)

Converting X to appropriate Datatype...

In [173]:
X_tensor_train = torch.from_numpy(X_train)
X_tensor_test = torch.from_numpy(X_test)
y_tensor_train = torch.from_numpy(y_train)
y_tensor_test = torch.from_numpy(y_test)

In [174]:
X_tensor_train = torch.tensor(X_tensor_train,dtype = torch.float32)
X_tensor_test = torch.tensor(X_tensor_test,dtype = torch.float32)
y_tensor_train = y_tensor_train.type(torch.LongTensor)
y_tensor_test = y_tensor_test.type(torch.LongTensor)

/tmp/ipykernel_100/3712994864.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_tensor_train = torch.tensor(X_tensor_train,dtype = torch.float32)
/tmp/ipykernel_100/3712994864.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_tensor_test = torch.tensor(X_tensor_test,dtype = torch.float32)


Pytorch needs the true labels to be in the range N-1. Thus, substract 1 one from each label.

In [175]:
y_tensor_train = y_tensor_train-1
y_tensor_test = y_tensor_test-1

In [176]:
print('training shape X is {}, test shape X is{}'.format(X_tensor_train.shape,X_tensor_test.shape))

training shape X is torch.Size([160, 1, 224, 224]), test shape X istorch.Size([40, 1, 224, 224])


Setup custom dataset class

In [177]:
class SimpleDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        X = self.X[idx]
        y = self.y[idx]
        return X, y

In [178]:
import torch.nn as nn

In [179]:
dataset_train = SimpleDataset(X_tensor_train,y_tensor_train)
dataloader_train = DataLoader(dataset_train, batch_size=32, shuffle=True)

dataset_test = SimpleDataset(X_tensor_test,y_tensor_test)
dataloader_test = DataLoader(dataset_test, batch_size = 32,shuffle=True)
criterion = nn.CrossEntropyLoss()

Setting up the CNN.

In [180]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3)
        self.conv2 = nn.Conv2d(32, 64, 3)
        self.fc1 = nn.Linear(774400, 128)
        self.fc2 = nn.Linear(128, 3)
        self.pool = nn.MaxPool2d(2)
        
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x


In [181]:
SalienceNet = CNN()

In [182]:
def analysis(net, loss_fn, dataloader_train,dataloader_test,
          epoch=100, learning_rate=0.01):
    optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)#,weight_decay=1e-4)
    train_loss = []
    acc=[]
    for i in range(epoch):   
        net.train()
        correct, total = 0, 0
        loss_count = 0
        for data, label in dataloader_train:
            data, label = data.to('cpu'), label.to('cpu')
        #make prediction
            output = net(data)

            # Zero out the gradients
            optimizer.zero_grad()

            # Calculate loss.
            loss = criterion(output,label)#torch.max(label, 1)[0])

            # Backpropagation and gradient update.
            loss.backward()# Calculate gradients. #retrain_graph=True

            optimizer.step() # Apply gradient udpate.

            loss_count += loss.item()
            _, predicted = torch.max(output, 1)
            print(predicted, torch.max(label,1)[0])
            total += label.size(0)
            correct += (predicted == torch.max(label,1)[0]).sum().item()
          #  if (i + 1) % (epoch // 5) == 0:
           #     print(f'iteration {i + 1}/{epoch} | loss: {loss.item():.3f}')
    acc.append(correct/total)
    train_loss.append(loss_count/len(dataloader_train))
    
    net.eval()
    test_loss = []
    test_acc = []
    correct_test, total_test = 0,0
    for data, labels in dataloader_test:
        data, labels = data.to('cpu'), labels.to('cpu')
        
        outputs = net(data)
        _, pred_test = torch.max(outputs,1)
        total_test += labels.size(0)
        print(total_test)
        correct_test += (pred_test == torch.max(labels,1)[0]).sum().item()
        print(correct_test)
        test_l = criterion(outputs,torch.max(labels, 1)[0])
        test_loss.append(test_l.item())
    test_acc.append(correct_test/total_test)
    print(acc,test_acc)
    return acc,total,correct,train_loss,test_acc


EMOTIC LOSS FUNCTION

In [202]:
class ContinuousLoss_L2(nn.Module):
  ''' Class to measure loss between continuous emotion dimension predictions and labels. Using l2 loss as base. '''
  def __init__(self, margin=1):
    super(ContinuousLoss_L2, self).__init__()
    self.margin = margin
  
  def forward(self, pred, target):
    labs = torch.abs(pred - target)
    loss = labs ** 2 
    loss[ (labs < self.margin) ] = 0.0
    return loss.sum()

In [183]:
import matplotlib.pyplot as plt

In [190]:
#i=0
#while i < 1:
 #   for x,y in dataloader_train:
  #      out=SalienceNet(x)
   #     loss = criterion(out,y.float())
    #    print(loss, out, y)
     #   i=i+1

In [192]:
testtest=X_tensor_train[0:1,:]

In [193]:
testtest.shape

torch.Size([1, 1, 224, 224])

In [195]:
out = SalienceNet(testtest)
out

tensor([[ 0.0053,  0.0434, -0.0565]], grad_fn=<AddmmBackward0>)

In [203]:
cont_loss = ContinuousLoss_L2()

In [204]:
cont_loss_batch = cont_loss(out * 10, y_tensor_train[0:1,:] * 10)

In [208]:
out*10

tensor([[ 0.0526,  0.4338, -0.5647]], grad_fn=<MulBackward0>)

In [205]:
cont_loss_batch

tensor(8607.2871, grad_fn=<SumBackward0>)

In [197]:
loss = criterion(out,y_tensor_train[0:1,:].float())

In [198]:
loss

tensor(17.5834, grad_fn=<DivBackward1>)

In [199]:
_, predicted = torch.max(out, 1)

In [201]:
_, predicted

(tensor([0.0434], grad_fn=<MaxBackward0>), tensor([1]))

In [184]:
np.random.seed(1)
torch.manual_seed(1)

acc,total,correct,train_loss,test_acc=analysis(net=SalienceNet,loss_fn=criterion,dataloader_train=dataloader_train,
                  dataloader_test=dataloader_test,epoch=1)



IndexError: Target 6 is out of bounds.

In [39]:
train_loss

[2.505991776784261]

In [41]:
i=[]

i.append(6/3)
i

[2.0]

In [81]:
total,correct = 0,0
i=0
loss = []
while i < 5:
    for x,y in dataloader_train:
        output = SalienceNet(x)
        _, predicted = torch.max(output, 1)
        #predicted = torch.tensor()
        total += y.size(0)
        correct += (predicted == torch.max(y,1)[0]).sum().item()
        print(predicted,torch.transpose(y,1,0))
        y_I = torch.max(y,1)[0]
        i=i+1
loss.append(correct/total)
print(loss)

tensor([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4]) tensor([[5, 6, 3, 5, 6, 5, 5, 5, 5, 5, 5, 1, 4, 6, 6, 4, 5, 4, 4, 3, 6, 4, 6, 6,
         5, 5, 4, 3, 4, 5, 4, 4]])
tensor([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4]) tensor([[6, 5, 4, 3, 6, 7, 4, 4, 4, 5, 5, 9, 6, 7, 5, 4, 6, 6, 6, 6, 6, 7, 2, 5,
         6, 6, 7, 5, 4, 7, 5, 6]])
tensor([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]) tensor([[4, 1, 5, 6, 3, 5, 6, 7, 5, 6, 5, 2, 4, 5, 5, 5]])
tensor([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4]) tensor([[6, 5, 5, 5, 5, 3, 7, 3, 5, 4, 6, 5, 3, 5, 5, 7, 4, 9, 6, 7, 3, 4, 6, 5,
         1, 7, 5, 6, 5, 4, 4, 5]])
tensor([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4]) tensor([[6, 6, 5, 6, 6, 4, 5, 7, 7, 5, 6, 5, 4, 4, 6, 1, 4, 5, 6, 5, 6, 6, 2, 5,
   

In [77]:
_, predicted

(tensor([0.1476, 0.1476, 0.1476, 0.1476, 0.1476, 0.1476, 0.1476, 0.1476, 0.1476,
         0.1476, 0.1476, 0.1476, 0.1476, 0.1476, 0.1476, 0.1476],
        grad_fn=<MaxBackward0>),
 tensor([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]))

In [83]:
y_I

tensor([4, 4, 6, 4, 5, 6, 6, 5, 5, 4, 5, 5, 4, 4, 6, 2])

In [88]:
total

160

In [51]:
output

tensor([[-65.9044,  44.7948, -50.0664,  22.9917,  63.5114, -65.3450,  30.7097,
         130.8091, -44.3176,   9.8188],
        [-66.7599,  45.2335, -50.6327,  23.2442,  64.3278, -66.1742,  31.0289,
         132.4245, -45.0380,   9.9740],
        [-67.2260,  45.4627, -50.9415,  23.4104,  64.8238, -66.7101,  31.0802,
         133.4065, -45.5668,  10.1352],
        [-66.6873,  45.1616, -50.5517,  23.2404,  64.2822, -66.0941,  30.9580,
         132.2808, -45.0118,  10.0283],
        [-66.2129,  44.9116, -50.2500,  23.0817,  63.8136, -65.5273,  30.9220,
         131.2758, -44.4338,   9.8651],
        [-67.0469,  45.4192, -50.8229,  23.3484,  64.6289, -66.5578,  31.0135,
         133.0656, -45.3902,  10.1040],
        [-66.3755,  45.0062, -50.3480,  23.1280,  63.9697, -65.7896,  30.8553,
         131.6962, -44.7400,   9.9400],
        [-67.0683,  45.3656, -50.8393,  23.3909,  64.6840, -66.6029,  30.9773,
         133.1539, -45.5212,  10.1378],
        [-66.3048,  44.9367, -50.3148,  23.1251,

In [ ]:
correct

In [ ]:
total

In [ ]:
5/40

In [ ]:
for idx, (x,y) in enumerate(dataloader_train):
    print(y)
    

In [ ]:
acc,total,correct

In [ ]:
plt.plot(train_loss)
plt.xlabel('Iterations of gradient descent')
plt.ylabel('Cross entropy')
plt.show()